In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

In [2]:
os.chdir('D:/DS/Kaggle')

In [3]:
# Load data
pageviews = pd.read_csv('./pageviews.csv')
# content_category = pd.read_csv('./CONTENT_CATEGORY.csv')
# content_category_bottom = pd.read_csv('./CONTENT_CATEGORY_BOTTOM.csv')
# content_category_top = pd.read_csv('./CONTENT_CATEGORY_TOP.csv')
# device_data = pd.read_csv('./device_data.csv')
page = pd.read_csv('./PAGE.csv')
# site_id = pd.read_csv('./SITE_ID.csv')
conversiones = pd.read_csv('./conversiones.csv', dtype={'mes' : 'object', 'anio' : 'object', 'USER_ID' : 'object'})

In [4]:
conversiones.head()

,mes,anio,USER_ID
0,7.0,2018.0,1410.0
1,8.0,2018.0,10755.0
2,8.0,2018.0,8270.0
3,10.0,2018.0,7558.0
4,9.0,2018.0,10731.0


###  Procesamiento archivo conversiones

Elimino el '.0' de los datos de conversiones

In [11]:
conversiones['mes'] = conversiones['mes'].str.replace('\.0', '')
conversiones['anio'] = conversiones['anio'].str.replace('\.0', '')
conversiones['USER_ID'] = conversiones['USER_ID'].str.replace('\.0', '')
conversiones['conversion'] = 1
conversiones.head()

,mes,anio,USER_ID,conversion
0,7,2018,1410,1
1,8,2018,10755,1
2,8,2018,8270,1
3,10,2018,7558,1
4,9,2018,10731,1


### Procesamiento archivo pageviews
Me quedo con los datos a analizar: PAGE y la fecha del evento
Crear columnas anio y mes

In [7]:
pageviews = pageviews.loc[:,['USER_ID', 'FEC_EVENT','PAGE']]
pageviews['FEC_EVENT'] = pd.to_datetime(pageviews['FEC_EVENT'])
pageviews['anio'] = pageviews['FEC_EVENT'].dt.year
pageviews['mes'] = pageviews['FEC_EVENT'].dt.month
pageviews.head()

,USER_ID,FEC_EVENT,PAGE,anio,mes
0,0,2018-03-30 07:35:48,1,2018,3
1,0,2018-03-30 07:35:52,2,2018,3
2,0,2018-03-30 07:36:11,3,2018,3
3,0,2018-03-30 07:36:16,4,2018,3
4,0,2018-03-30 07:41:38,5,2018,3


### Merge datasets
Hago un join y lleno con 0 en los casos que no hubo conversion

In [16]:
data = pd.merge(pageviews, conversiones, how='left', left_on = ['mes','anio','USER_ID'], right_on = ['mes','anio','USER_ID'])
data['conversion'] = data['conversion'] .replace(np.nan, 0)
data.head()

,USER_ID,FEC_EVENT,PAGE,anio,mes,conversion
0,0,2018-03-30 07:35:48,1,2018,3,0.0
1,0,2018-03-30 07:35:52,2,2018,3,0.0
2,0,2018-03-30 07:36:11,3,2018,3,0.0
3,0,2018-03-30 07:36:16,4,2018,3,0.0
4,0,2018-03-30 07:41:38,5,2018,3,0.0
